In [ ]:
import sqlite3

'''
一共開 3 個 DB
1. 存 NER 結果
2. 存 POS 結果
3. 存 RE 結果: 含網址、Email、Username、(行動、市內)電話、IP
4. 存
'''

con_NER = sqlite3.connect("JudicialAnalysis_R_NER.db")
cur_NER = con_NER.cursor()

con_POS = sqlite3.connect("JudicialAnalysis_R_POS.db")
cur_POS = con_POS.cursor()

con_RE = sqlite3.connect("JudicialAnalysis_R_RE.db")
cur_RE = con_RE.cursor()

con_TIME = sqlite3.connect("JudicialAnalysis_R_TIME.db")
cur_TIME = con_TIME.cursor()

In [ ]:
# 創建 Table
new = 0
if new:
    cur_NER.execute("CREATE TABLE results(jid, date, keyword_type, keywords)")
    con_NER.commit()
    con_NER.close()
    
    cur_POS.execute("CREATE TABLE results(jid, date,  keyword_type, keywords)")
    con_POS.commit()
    con_POS.close()
    
    cur_RE.execute("CREATE TABLE results(jid, date, keyword_type, keywords)")
    con_RE.commit()
    con_RE.close()
    
    cur_TIME.execute("CREATE TABLE results(jid, start_timestamp, lenth_raw, lenth_u, elapsed_time_RE, elapsed_time_WS, elapsed_time_POS, elapsed_time_NER)")
    con_TIME.commit()
    con_TIME.close()

In [ ]:
# 把原本裁判書檔按順序反過來
# 從 2023年12月開始
reverse = 0 # 已經做過就不用再做了
if reverse:
    import os 

    source_file = os.path.join(
            'C:\\', '課程', '交大','論文','案件整理',
            'json_files.txt'
            )

    with open(source_file, 'r', encoding='utf-8') as f:
        # 2022~2023年判決書有211萬7,454個json檔
        # 故一次讀一個處理，減少記憶體占用
        json_files_R = f.read().strip().split('\n')

    json_files_R.sort(reverse=True)
    json_files_R = [f for f in json_files_R if os.path.isfile(f)]

    source_file_R = os.path.join(
            'C:\\', '課程', '交大','論文','案件整理',
            'json_files_R.txt'
            )
    
    json_files_R_STR = '\n'.join(json_files_R).strip()
    
    with open(source_file_R, 'w', encoding='utf-8') as f:
        # 2022~2023年判決書有211萬7,454個json檔
        # 故一次讀一個處理，減少記憶體占用
        f.read().strip().split('\n')

In [ ]:
import re
from valid_IP import valid_IPv4s, valid_IPv6s

# python3 以後 str.isalpha() 跟 str.isalnum() 對於中文都視作 unicode，所以都會 return True，故轉換成 ASCII 再判斷
def is_pure_english(text):
    return text.isalpha() and all(ord(c) < 128 for c in text)

# 用來抓網址s、Domains
def extract_urls(text):
    urls = []
    # 匹配包含 http:// 或 https:// 的 URL
    # 以及試圖匹配裸域名（如 example.com），排除郵箱地址
    patterns = [
        r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',
        r'\b(?<![@])(([\w-]+\.)+[a-zA-Z]{2,})\b'
    ]
    for url_pattern in patterns:
        urls += re.findall(url_pattern, text)
    
    # 從匹配結果中提取 URL，忽略分組捕獲
    extracted_urls = [url[0] if isinstance(url, tuple) else url for url in urls]

    # 去重並排序
    urls = list(set(extracted_urls))
    urls.sort()
    return urls

# 用來抓 Emails
def extract_emails(text):
    email_pattern = r'[\w.-]+@[\w.-]+'
    emails = re.findall(email_pattern, text)
    emails = list(set(emails))
    emails.sort()
    return emails

# 用來抓 Usernames
# \w : 包括(alphanumeric) [0-9a-zA-Z_]
# 後來發現 \w 在 python3 是 unicode 也支持中文
# 解決方法: 要在 re.findall 第三個參數加入 re.A 恢復使用 ASCII，例如: re.findall(r"\w", text, re.A))
# 我是乾脆直接指定 [0-9a-zA-Z_]
def extract_usernames(text):
    # 更新後的用戶名模式
    # 正向後瞻和正向前瞻斷言確保前後不是字母或數字
    # 添加對「@用戶名」特殊情況的處理
    username_patterns = [
        r'(?<![0-9a-zA-Z])@?[a-zA-Z][0-9a-zA-Z_.\-]+(?![0-9a-zA-Z])',  # 基本用戶名模式，前後不接字母或數字
        r'「(@?[a-zA-Z][0-9a-zA-Z_.\- ]+)」'  # 在「」中的用戶名
    ]

    usernames = []
    for pattern in username_patterns:
        usernames += re.findall(pattern, text)

    # 去重並排序
    usernames = list(set(usernames))
    usernames.sort()
    return usernames

# 用來抓手機號碼
def extract_mobilephones(text):
    # 手機可能有 - 也可能沒有 -
    # 國際號碼可能有 886
    cellphones = []
    # 定義手機號碼模式，包括正向前瞻和正向後瞻斷言
    # 這裡使用 (?<![0-9]) 斷言確保模式前面不是數字
    # 使用 (?![0-9]) 斷言確保模式後面不是數字
    cellphone_patterns = [
        r'(?<![0-9])(\+?886\d{3}-?\d{3}-?\d{3})(?![0-9])', # 對於+886 (台灣國碼) 格式的號碼
        r'(?<![0-9])(0\d{3}-?\d{3}-?\d{3})(?![0-9])' # 對於本地格式的號碼 (0開頭)
    ]
    cellphones = list(set(cellphones))
    cellphones.sort()
    return cellphones

# 用來抓市內電話號碼
# 規則參考: https://zh.wikipedia.org/zh-tw/%E4%B8%AD%E8%8F%AF%E6%B0%91%E5%9C%8B%E9%9B%BB%E8%A9%B1%E8%99%9F%E7%A2%BC
def extract_telephones(text):
    telephone_pattern = r'(?<![0-9])\(?0[2-8]\d{0,3}\)?-?\d{1,4}-?\d{4}(?![0-9])'
    telephones = re.findall(telephone_pattern, text)
    telephones = list(set(telephones))
    telephones.sort()
    return telephones

# 用來抓 IP
def extract_ip(text):
    v4a = valid_IPv4s(text)
    v6a = valid_IPv6s(text)
    
    # 結合兩者結果
    ips = list(set(v4a + v6a))
    ips.sort()
    return ips

# 用來初步抓罪名
def extract_crime(text):
    crimes = []
    patterns = [
        r'違反(\S{2,20})案',  
        r'違反(\S{2,20})等案', 
        #r'違反(\S{2,20})罪',
        r'因(\S{2,20})案', 
        r'因(\S{2,20})等案', 
        #r'因(\S{2,20})罪', 
        r'號(\S{2,20})案',
        #r'號(\S{2,20})等', 
        #r'號(\S{2,20})罪', 
        r'被告(\S{2,20})案', 
        #r'被告(\S{2,20})等'
        #r'被告(\S{2,20})罪'
    ]
    for pattern in patterns:
        crimes += re.findall(pattern, text)
    
    # 這裡整理一下，把可能太長的去掉(到等、案件、罪就停止)
    stop_words = ['等', '案']
    drop_words = [
        '刑事判決', '聲請', '起訴', '刑責', '檢察', '上開', '事實', '被告', '(', ')', '判決', '如下', '主文', '之犯', '特定犯',
        '法理', '係', '可疑', '從而', '當場', '諭知', '。', '行經', '裁定', '認無', '觀察', '處斷', '「', '」', '決定',
    ]
    fix_crimes = []
    for c in crimes:
        c = c.strip().replace('違反', '').replace('因', '').replace('、', '').replace(',', '').replace(':', '').replace('：', '')
        for sw in stop_words:
            idx_stop = c.find(sw)
            if idx_stop > 0:
                c = c[:idx_stop]
        c = c.strip().replace('等', '')
        if len(c) > 1:
            if not str(c[0]).isdigit() and not is_pure_english(c[0]):
                check_to_drop = 0
                for dw in drop_words:
                    if dw in c:
                        check_to_drop += 1
                if check_to_drop == 0:
                    fix_crimes.append(c)
    crimes = fix_crimes + []
    
    crimes = list(set(crimes))
    crimes.sort()
    
    return crimes

re_extractors = {
    'crimes' : extract_crime,
    'urls' : extract_urls,
    'emails' : extract_emails,
    'usernames' : extract_usernames,
    'mobilephones' : extract_mobilephones,
    'telephones' : extract_telephones,
    'ips' : extract_ip,
}

In [ ]:
from ElementExtractor import Judicial_Parser
import os, time, traceback, json, re

In [ ]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER

In [ ]:
# Downloads to ./data.zip (2GB) and extracts to ./data/
# data_utils.download_data_url("./") # iis-ckip
#data_utils.download_data_gdown("./") # gdrive-ckip 已下載就不用再下載

In [ ]:
# To use GPU:
#    1. Install tensorflow-gpu (see Installation)
#    2. Set CUDA_VISIBLE_DEVICES environment variable, e.g. os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#    3. Set disable_cuda=False, e.g. ws = WS("./data", disable_cuda=False)

# 採坑心得
# 新版已經沒有 tensorflow-gpu
# 我使用 python 3.7 的環境 (conda create -n ckipcore python=3.7)
# 先安裝 pip install -U ckiptagger[tf,gdown]
# 再改用 Conda install tensorflow 去安裝匹配的新版
# 搭配 disable_cuda=False 參數，從 windows 工作管理員之效能頁籤看起來，確實有使用 GPU

ws = WS("./data", disable_cuda=False)
pos = POS("./data", disable_cuda=False)
ner = NER("./data", disable_cuda=False)

In [ ]:
# 簡繁轉換器 (暫時用不到)
# import opencc
# converter = opencc.OpenCC('t2s.json')

In [ ]:
from IPython.display import clear_output

start_time = time.time()
print(f'[現在時間] {time.strftime("%Y-%m-%d %H:%M:%S")}')

source_file = os.path.join(
        'C:\\', '課程', '交大','論文','案件整理',
        'json_files_R.txt'
        )
    
count_file_done_num = 0
clear_output()

# 多工時使用
skip = 0 # 要跳過從第幾開始

print('-'*50)
print(f'>>開始分析...')
print(f'[當前已分析數量] {count_file_done_num}')

with open(source_file, 'r', encoding='utf-8') as f:
    # 2022~2023年判決書有211萬7,454個json檔
    # 故一次讀一個處理，減少記憶體占用
    raw_data = ''
    
    while 1: 
        # 開始逐案讀取資料分析
        file = f.readline()
        count_file_done_num += 1
        
        if '刑事' in file:
            # 篩掉一些比較非刑案的
            if not ('簡易' in file or '民事' in file or '補償' in file or '憲法' in file or '商業' in file):
                # 開始讀取裁判書內容
                file = file.strip()

                # 顯示進度
                temp_time = time.time()
                elapsed_time = temp_time - start_time
                hours = int(elapsed_time // 3600)  # 計算小時數
                minutes = int((elapsed_time % 3600) // 60)  # 計算分鐘數
                seconds = int(elapsed_time % 60)  # 計算剩餘秒數
                
                clear_output()
                print('-'*50)
                print(f'>>開始分析...')
                print(f'[當前已分析數量] {count_file_done_num}')
                print(f'[累計耗時] {hours}:{minutes}:{seconds}')
                print(f'[當前檔案] {file}')
                
                # 確定 file 存在
                if not os.path.isfile(file):
                    print(f'[並非正常檔案] 跳過...')
                    time.sleep(1)
                    continue
                
                try:
                    with open(file, 'r', encoding='utf-8') as jf:
                        json_data = json.load(jf)
                except:
                    x = traceback.format_exc()
                    print(x)
                    time.sleep(3)
                    continue
                    
                # 資料清洗
                parser = Judicial_Parser(json_data)
                
                jid = parser.jid.strip().replace(' ', '')
                date = parser.dateSTR.strip().replace(' ', '')
                raw = parser.raw
                text = parser.text
                text_u = parser.text_u
                
                # 確定是否已經在資料庫內
                check_res = cur_POS.execute(f"SELECT jid FROM results WHERE jid='{jid}'")
                temp = check_res.fetchone()
                if temp:
                    print(f'[JID] {jid}')
                    print('>>此JID發現資料庫已有，跳過')
                    continue
                    
                # 抓字數
                lenth_raw = len(raw)
                lenth_u = len(text_u)
                
                print(f'[lenth_raw] {lenth_raw}')
                print(f'[lenth_u] {lenth_u}')
                    
                print('-'*50)
                print(text_u[:500])
                print('-'*50)
                
                # 紀錄開始時間
                start_time = time.time()
                
                # 沒有在資料庫內才處理
                
                '''
                ------------------------------------------------------------------
                RE 擷取
                ------------------------------------------------------------------
                '''
                 # Input text
                print('[RE] 分析中')
                # 擷取 RE 結果
                re_dict = dict()
                for re_type, re_func in re_extractors.items():
                    re_temp_result = re_func(text_u)
                    if re_temp_result:
                        re_dict[re_type] = re_temp_result
                    print(f'>> [{re_type}] {re_temp_result}')
                
                elapsed_time_RE = time.time() - start_time
                
                '''
                ------------------------------------------------------------------
                POS & NER 擷取
                ------------------------------------------------------------------
                '''
                # Input text
                print('-'*50)
                print('[WS] 分析中')
                sentence_list = text_u.split('。')
                word_sentence_list = ws(
                    sentence_list,
                    # sentence_segmentation = True, # To consider delimiters
                    # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
                    # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
                    # coerce_dictionary = dictionary2, # words in this dictionary are forced
                )
                
                elapsed_time_WS = time.time() - start_time
                
                print('[POS] 分析中')
                pos_sentence_list = pos(word_sentence_list)
                
                # 整理 POS 結果
                pos_dict = dict()
                for idx, words in enumerate(word_sentence_list):
                    for idx2, word in enumerate(words):
                        temp_type = pos_sentence_list[idx][idx2]
                        word = word.strip().replace(' ', '').replace(':', '').replace(',', '')
                        pos_dict.setdefault(temp_type, set())
                        pos_dict[temp_type].add(word)
                        
                elapsed_time_POS = time.time() - start_time
                
                
                print('[NER] 分析中')
                entity_sentence_list = ner(word_sentence_list, pos_sentence_list)
                
                # 擷取 NER 結果
                ner_dict = dict()
                for e in entity_sentence_list:
                    for ee in e:
                        ner_type = ee[2]
                        ner_dict.setdefault(ner_type, set())
                        temp_ner_result = ee[3].strip().replace(' ', '').replace(':', '').replace(',', '')
                        ner_dict[ner_type].add(temp_ner_result)
                        
                elapsed_time_NER = time.time() - start_time        
                        
                print('-'*50)
                
                '''
                ------------------------------------------------------------------
                寫入 DB Table
                ------------------------------------------------------------------
                '''
                print('-'*50)
                print('>>準備寫入 RE 結果')
                for re_type, re_keywords in re_dict.items():
                    values_str = ','.join(re_keywords)
                    to_db_content = "INSERT INTO results VALUES(?, ?, ?, ?)" # ? 是一個參數占位符，代表將要插入的值。
                    cur_RE.execute(to_db_content, (jid, date, re_type, values_str))
                con_RE.commit()
                
                print('-'*50)
                print('>>準備寫入 POS 結果')
                for temp_type, temp_keywords in pos_dict.items():
                    temp_keywords = {tk.strip().replace(' ', '').replace(',', '').replace(':', '') for tk in temp_keywords if len(tk.strip().replace(' ', '').replace(',', '').replace(':', '')) > 1}
                    temp_keywords = list(temp_keywords)
                    temp_keywords.sort()
                    values_str = ','.join(temp_keywords)
                    to_db_content = "INSERT INTO results VALUES(?, ?, ?, ?)" # ? 是一個參數占位符，代表將要插入的值。
                    cur_POS.execute(to_db_content, (jid, date, temp_type, values_str))
                con_POS.commit()
                
                print('-'*50)
                print('>>準備寫入 NER 結果')
                for temp_type, temp_keywords in ner_dict.items():
                    temp_keywords = {tk.strip().replace(' ', '').replace(',', '').replace(':', '') for tk in temp_keywords if len(tk.strip().replace(' ', '').replace(',', '').replace(':', '')) > 1}
                    temp_keywords = list(temp_keywords)
                    temp_keywords.sort()
                    values_str = ','.join(temp_keywords)
                    to_db_content = "INSERT INTO results VALUES(?, ?, ?, ?)" # ? 是一個參數占位符，代表將要插入的值。
                    cur_NER.execute(to_db_content, (jid, date, temp_type, values_str))
                con_NER.commit()
                
                print('-'*50)
                print('>>準備寫入 Elapsed_time 結果')
                to_db_content = "INSERT INTO results VALUES(?, ?, ?, ?, ?, ?, ?, ?)" # ? 是一個參數占位符，代表將要插入的值。
                cur_TIME.execute(to_db_content, (jid, start_time, lenth_raw, lenth_u, elapsed_time_RE, elapsed_time_WS, elapsed_time_POS, elapsed_time_NER))
                con_TIME.commit()
                
                print('-'*50)
                print('>>完成!!!')
        
# 關閉 DB占用
con_RE.close()
con_POS.close()
con_NER.close()
con_TIME.close()

# 顯示統計結果
end_time = time.time()
elapsed_time = end_time - start_time
hours = int(elapsed_time // 3600)  # 計算小時數
minutes = int((elapsed_time % 3600) // 60)  # 計算分鐘數
seconds = int(elapsed_time % 60)  # 計算剩餘秒數
print(f'[累計耗時] {hours}:{minutes}:{seconds}')